In [4]:
import tarfile
import os
from s3_utilities import get_s3_connection_profile
import boto3, botocore

In [3]:
cat ./images.tar.parta* > ./images.tar.gz

In [5]:
filename = './images.tar.gz'
extract_path = '.'

with tarfile.open(filename) as f:
    f.extractall(extract_path)
    f.close()

/usr/lib64/python3.11/tarfile.py:2282: RuntimeWarning: The default behavior of tarfile extraction has been changed to disallow common exploits (including CVE-2007-4559). By default, absolute/parent paths are disallowed and some mode bits are cleared. See https://access.redhat.com/articles/7004769 for more details.
  warnings.warn(


In [ ]:
def delete_folder_if_exists(bucket, folder_name: str):
    prefix = folder_name.rstrip('/') + '/'
    objects = list(bucket.objects.filter(Prefix=prefix))

    if not objects:
        print(f"No folder named '{folder_name}' found in bucket '{bucket.name}'.")
        return

    print(f"Deleting folder '{folder_name}' and its contents in bucket '{bucket.name}'...")

    for obj in objects:
        print(f"Deleting: {obj.key}")
        obj.delete()

    print("Folder deleted successfully.")

In [5]:
s3_profile = get_s3_connection_profile()

local_images_directory = "./downloaded_images"
 
session = boto3.session.Session(
        aws_access_key_id=s3_profile.access_key,
        aws_secret_access_key=s3_profile.secret_key
      )

s3_resource = session.resource('s3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=s3_profile.endpoint_url,
    region_name=s3_profile.region
    )
 
edb_bucket = s3_resource.Bucket(s3_profile.bucket_name)

In [ ]:
delete_folder_if_exists(edb_bucket, s3_profile.recommender_images_path);

In [ ]:
images_uploaded = 0
print(f"Uploading images to s3 bucket {s3_profile.endpoint_url}:{s3_profile.bucket_name} ... ", end="")

for root, dirs, files in os.walk(local_images_directory):
    for filename in files:
        file_path = os.path.join(root, filename)
        object_key = os.path.join(s3_profile.recommender_images_path, filename)
        edb_bucket.upload_file(file_path, object_key)
        images_uploaded += 1
        if images_uploaded % 100 == 0:
            print(f"{images_uploaded} ", end="")

print("Upload completed.")